In [1]:
import os
import sys

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, unique, vstack, join, Column

sys.path.insert(0, '..')
from surface_brightness import surface_brightness


/Users/Daniel/anaconda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
#Input / Output Information
input_data_table = 'Rigault 2015 data table.csv'
fits_directory = '/Volumes/Media RED/fits files/Rigault 15 Supernova' #The directory to look for .fits files in
output_file_name = "Rigault 2015" #The output file name without an extension


In [ ]:
if os.path.exists(fits_directory) and os.listdir(fits_directory):
    cord_dict = {} #for supernova coordinates
    red_dict = {}#for supernova redshifts
    
    rigault_15_table = Table.read(input_data_table, format = 'csv')
    for row in rigault_15_table:
        red_dict[row['sn']] = row['z']
        cord_dict[row['sn']] = SkyCoord(row['RA'] + ' ' + row['Dec'], unit = (u.hourangle, u.deg))
    
    brightness = surface_brightness(cord_dict, red_dict)
    
    print('Calculating NUV values using a 2kpc radius aperture\n', flush = True)
    nuv_2kpc = brightness.create_tables('NUV', fits_directory, 2, show_list = True)
    
    print('\nCalculating FUV values using a 2kpc radius aperture\n', flush = True)
    fuv_2kpc = brightness.create_tables('FUV', fits_directory, 2, show_list = True)

elif not os.path.exists(fits_directory):
    print('Specified directory of .fits files does not exist')
    
else:
    print('Specified directory of .fits files is empty')

Calculating NUV values using a 2kpc radius aperture

290 : /Volumes/Media RED/fits files/Rigault 15 Supernova/2415374296051351552/MISDR1_33982_0341-nd-int.fits
289 : /Volumes/Media RED/fits files/Rigault 15 Supernova/2416183536609394688/MISDR1_10093_0354-nd-int.fits
288 : /Volumes/Media RED/fits files/Rigault 15 Supernova/2422868567306272768/MISDR1_18848_0459-nd-int.fits
287 : /Volumes/Media RED/fits files/Rigault 15 Supernova/2482506077996843008/NGA_NGC0541-nd-int.fits
286 : /Volumes/Media RED/fits files/Rigault 15 Supernova/2487467074461368320/NGA_Arp105-nd-int.fits
285 : /Volumes/Media RED/fits files/Rigault 15 Supernova/2502561170087477248/NGA_UGC1922-nd-int.fits
284 : /Volumes/Media RED/fits files/Rigault 15 Supernova/2836671967443025920/MISDR2_00589_0559-nd-int.fits
283 : /Volumes/Media RED/fits files/Rigault 15 Supernova/2906442577295179776/MISWZS22_20251_0162-nd-int.fits
282 : /Volumes/Media RED/fits files/Rigault 15 Supernova/2906653683527712768/MISWZS22_20672_0261-nd-int.fits

In [ ]:
    nuv_2 = nuv_2kpc
    fuv_2 = fuv_2kpc
    
    merged_table = join(nuv_2[0], fuv_2[0], join_type = 'outer', keys = 'sn')
    merged_table.rename_column('NUV 2kpc Exposure Time', 'NUV Exposure Time')
    merged_table.rename_column('FUV 2kpc Exposure Time', 'FUV Exposure Time')
    merged_table.rename_column('Redshift_1', 'Redshift')
    merged_table.rename_column('Redshift Error_1', 'Redshift Error')
    
    for row in merged_table:
        if str(row["Redshift"]) == "--":
            row["Redshift"] = row["Redshift_2"]
            row["Redshift Error"] = row["Redshift Error_2"]
    
    merged_table.remove_columns(["Redshift_2", "Redshift Error_2"])